In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

# Drawing

In [2]:
boiling_plan_df = pd.read_csv('2020.12.13 boiling_plan.csv')
boiling_plan_df['sku'] = boiling_plan_df['sku'].apply(cast_sku)
# boiling_plan_df['boiling'] = boiling_plan_df['boiling'].apply(cast_boiling)
boiling_plan_df['boiling'] = boiling_plan_df['sku'].apply(lambda sku: sku.boilings[0])
boiling_plan_df.sort_values(by='id')
boiling_plan_df['speed'] = boiling_plan_df['sku'].apply(lambda sku: sku.packing_speed)
boiling_plan_df['bff'] = boiling_plan_df['sku'].apply(lambda sku:sku.boiling_form_factors[0])

boiling_plan_df.at[:2, 'packing_team_id'] = 1
boiling_plan_df.at[2:, 'packing_team_id'] = 2

boiling_plan_df.head()

,id,boiling,sku,kg,speed,bff,packing_team_id
0,1.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0
2,3.0,<Boiling 6>,<SKU 23>,192.8,960,<BoilingFormFactor 1>,2.0
3,3.0,<Boiling 6>,<SKU 26>,134.7,960,<BoilingFormFactor 1>,2.0
4,3.0,<Boiling 6>,<SKU 46>,153.6,960,<BoilingFormFactor 1>,2.0


In [3]:
boiling_plan_df['boiling_type'] = boiling_plan_df['boiling'].apply(lambda b: b.boiling_type)
boiling_plan_df = boiling_plan_df[boiling_plan_df['boiling_type'] == 'water']
boiling_plan_df.head()

,id,boiling,sku,kg,speed,bff,packing_team_id,boiling_type
0,1.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,water
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,water
2,3.0,<Boiling 6>,<SKU 23>,192.8,960,<BoilingFormFactor 1>,2.0,water
3,3.0,<Boiling 6>,<SKU 26>,134.7,960,<BoilingFormFactor 1>,2.0,water
4,3.0,<Boiling 6>,<SKU 46>,153.6,960,<BoilingFormFactor 1>,2.0,water


In [4]:
def add_push_with_props(parent, block):
    return add_push(parent, block, new_props=block.props.get_all_props())

In [5]:
mode = 'prod'
maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

root = make_schedule(boiling_plan_df)

In [6]:
root

root (0, 3] x (0, 203]
  boiling (0, 3] x (48, 146]
    pouring (0, 2] x (48, 75]
      block (0, 1] x (48, 75]
        termizator (0, 1] x (48, 54]
        pouring_name (0, 1] x (54, 75]
      block (1, 2] x (48, 75]
        pouring_and_fermenting (1, 2] x (48, 56]
        soldification (1, 2] x (56, 63]
        cutting (1, 2] x (63, 70]
        pouring_off (1, 2] x (70, 73]
        extra (1, 2] x (73, 75]
    drenator (0, 1] x (75, 118]
    melting_and_packing (0, 3] x (118, 146]
      melting (0, 3] x (118, 146]
        labels (0, 1] x (118, 132]
          serving (0, 1] x (118, 124]
          melting_name (0, 1] x (124, 132]
        meltings (1, 2] x (118, 138]
          serving (1, 2] x (118, 124]
          full_melting_process (1, 2] x (124, 138]
            melting_process (1, 2] x (124, 138]
        coolings (2, 3] x (118, 146]
          cooling_process (2, 3] x (124, 146]
            start (2, 3] x (124, 132]
              cooling (2, 3] x (124, 128]
              cooling (2, 

In [7]:
with make('schedule'):
    make(h=1, visible=False, push_func=dummy_push_y)

    with make('header', h=1, index_width=3, push_func=dummy_push_y):
        make(h=1, size=1, text='График наливов', push_func=add_push)
        for i in range(288):
            cur_time = cast_time(i + cast_t('01:00'))
            if cur_time[-2:] == '00':
                make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
        
    for i in range(2):
        with make(f'cheese_maker_{i}', h=2, push_func=dummy_push_y):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(h=1, visible=False, push_func=dummy_push_y)
        
    with make('cleanings', h=2, push_func=dummy_push_y):
        for block in root['cleaning']:
            make(block, push_func=add_push_with_props)
    make(h=1, visible=False, push_func=dummy_push_y)

#     for i in range(2, 4):
#         with make(f'cheese_maker_{i}', h=2, push_func=dummy_push_y):
#             for block in root['boiling']:
#                 if block.props['pouring_line'] == str(i):
#                     make(block['pouring'], push_func=add_push_with_props)
#         make(h=1, visible=False, push_func=dummy_push_y)
    
    with make('header', h=1, index_width=4, push_func=dummy_push_y):
        for i in range(288):
            cur_time = cast_time(i + cast_t('07:00'))
            if cur_time[-2:] == '00':
                make(t=i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    with make('water_melting', h=4, push_func=dummy_push_y):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['melting'], push_func=add_push_with_props)
                
#     with make('water_packing', h=3, push_func=dummy_push_y):
#         for block in root['boiling']:
#             if block.props['boiling_type'] == 'water':
#                 make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
    
    # todo: make dynamic lines
#     n_lines = 5
#     melting_lines = [make(f'salt_melting_{i}', h=3, push_func=dummy_push_y).block for i in range(n_lines)]
    
#     # todo: hardcode, add empty elements for drawing not to draw melting_line itself
#     for b in melting_lines:
#         add_push(b, Block('block', visible=False))
        
#     for i, block in enumerate([b for b in root['boiling'] if b.props['boiling_type'] == 'salt']):
#         add_push_with_props(melting_lines[i % n_lines], block['melting_and_packing']['melting'])
    
#     with make('salt_packing', h=3, push_func=dummy_push_y):
#         for block in root['boiling']:
#             if block.props['boiling_type'] == 'salt':
#                 make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
row_root = maker.root

In [8]:
style = load_style()
init_sheet_func = init_sheet if mode == mode else init_template_sheet(template_fn=template_fn)
wb = draw_schedule(row_root, style, init_sheet_func=init_sheet_func)
wb.save('output.xlsx')
open_file_in_os('output.xlsx') 